In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope
from widget_code_input import WidgetCodeInput
from ipywidgets import Textarea
from iam_utils import *
import ase
from ase.io import read, write
from ase.calculators import lj, eam

In [ ]:
#### AVOID folding of output cell 

In [ ]:
%%html

<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:4000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [ ]:
data_dump = WidgetDataDumper(prefix="module_04")
display(data_dump)

In [ ]:
module_summary = Textarea("general comments on this module", layout=Layout(width="100%"))
data_dump.register_field("module-summary", module_summary, "value")
display(module_summary)

_Reference textbook / figure credits: Allen, Tildesley, Computer simulations of liquids, (2017), Chapter 1_

# Interatomic potentials

Interatomic potentials describe the energy (~stability) of a set of atoms - characterized by their chemical nature $a_i$ and Cartesian coordinates $\mathbf{r}_i$ - in terms of a model that describes their interactions. This potential $V(\{\mathbf{r}_i\})$ can be seen as an approximation of the quantum mechanical energy of the electrons in a material or molecule for a given position of the nuclei (the so-called Born-Oppenheimer approximation).

Many empirical forms have been proposed to model the interatomic potential. A typical potential might look something like

$$
V(\{\mathbf{r}_i\}) = \sum_{ij} \frac{Z_i Z_j}{|\mathbf{r}_i - \mathbf{r}_j|} - \sum_{ij} \frac{A}{|\mathbf{r}_i - \mathbf{r}_j|^6} + k \sum_{i,j \in \mathrm{bonds}} k (|\mathbf{r}_i - \mathbf{r}_j| - r_0)^2
$$

where you may recognize an electrostatic term, a dispersion interaction, and harmonic bonds that are usually chosen as a simple model of covalent bonds. 

Coulomb and dispersion forces are usually referred to as _non-bonded_ terms, in that they act between all pairs of atoms of a given kind. Harmonic springs terms (or angles, or dihedrals) are _bonded_ terms, that only act between selected groups of atoms that are chosen based on a predetermined topology of the covalent bonds.

All of the terms above are _pair potentials_ i.e. functions of just the distance between pairs of atoms. More complicated functional forms exist, as we shall see later.

<span style="color:blue">**01** How does the energy of a non-bonded and a bonded term in the potential change as the separation between two atoms tends to infinity? Can a harmonic bond ever truly dissociate? </span>

In [ ]:
ex01_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex01-answer", ex01_txt, "value")
display(ex01_txt)

An archetypal example of a non-bonded potential is the Lennard-Jones potential (if you are curious, you can read [the paper in which the general functional form was proposed](https://doi.org/10.1098/rspa.1924.0081)).
The LJ potential is a non-bonded pair potential $V(r)$ in which the attractive and repulsive parts are both algebraic functions of the interatomic separation, $A/r^m-B/r^n$. Usually $1/r^6$ is used for the attractive part (that physically corresponds to dispersion/van der Waals forces), and $1/r^{12}$ for the repulsive parts (which is chosen just to have a steep repulsive wall, and because back in the old days you could compute this just by squaring $1/r^6$, which was cheaper than recomputing another power. 

You can experiment below with the more general form of the potential,
$$
V(r) = \frac{A}{r^m} - \frac{B}{r^n}
$$
See how exponents and prefactors change the shape of the curve.

In [ ]:
def plot_LJ(ax, A, B, m, n, x_max = 3, y_min_relative = -1.5, y_max_relative = 2, n_points = 200):
    
    if (m != n):
        # min_pos and min_energy are max pos and max energy when n > m
        min_pos = np.exp((np.log(A) - np.log(B) + np.log(m) - np.log(n))/(m - n))
        min_energy = A / (min_pos ** m) - B / (min_pos ** n)
        min_energy = np.abs(min_energy)

        y_min = min_energy * y_min_relative
        y_max = min_energy * y_max_relative
    else:
        y_min, y_max = y_min_relative, y_max_relative
        
    grid = np.linspace(0, x_max, 200)[1:] # excluding 0
    curve = A / (grid ** m) - B / (grid ** n)
    
    ax.plot(grid, curve, color = 'red', linewidth = 2)
    ax.set_title(r"$V(r) = \frac{A}{r^m} - \frac{B}{r^n}$", fontsize = 15)
    ax.set_xlim([0, x_max])
    ax.set_ylim([y_min, y_max])
    ax.set_xlabel("r", fontsize = 15)
    ax.set_ylabel("V(r)", fontsize = 15)
    
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.tick_params(axis='both', which='minor', labelsize=12)
A = WidgetPlot(plot_LJ, WidgetParbox(A = (1.0, 0.1, 10, 0.1, r'A'),
                                       B = (1.0, 0.1, 10, 0.1, r'B'),
                                       m = (12, 1, 20, 1, r'm'),
                                       n = (6, 1, 20, 1, r'n'),
                                       ))
display(A)

The more common form to express the LJ potential is

$$
V(r) = 4\epsilon \left((\frac{\sigma}{r})^{12} - (\frac{\sigma}{r})^6\right).
$$

<span style="color:blue">**02** Compute analytically the equilibrium separation $r_0$ between two atoms (i.e. the position of the minimum in the $V(r)$ curve. What is the corresponding energy? </span>

In [ ]:
ex02_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex02-answer", ex02_txt, "value")
display(ex02_txt)

<span style="color:blue">**03** Now consider a set of four atoms arranged as a square with side $a$. Write a function that computes the total LJ potential for this structure, as a function of $a$. Inspect the curve as a function of $a$, using the sliders to select an appropriate range.</span>

_Take for simplicity $\epsilon=1$ and $\sigma=1$ (which is equivalent to writing the problem in natural units. You can write the summation as a sum over the pair distances, without writing explicitly the position of the particles._

In [ ]:
# set upt the code widget window
ex03_wci = WidgetCodeInput(
        function_name="total_LJ_square", 
        function_parameters="a",
        docstring="""
Computes the total LJ potential for the structure of four atoms arranged as a square with side a. 

:param a: side of the square
        
:return: the value of the total energy
""",
        function_body="""
# Write your solution. You can use np.sqrt(2) to get the value of sqrt(2)
# Note you can define a function inside a function body - use this to also write
# a function that computes the LJ potential at a given distance
import numpy as np

def compute_LJ(r):
    # computes the value of LJ potential depending on the distance r
    # use epsilon=sigma=1
    return ...
    
total_energy = 0.0  # write here a sum over the various interactions

return total_energy
"""
        )

data_dump.register_field("ex03-function", ex03_wci, "function_body")

def plot_total_energy(ax, x_min, x_max, y_min, y_max, n_points = 200):
    grid = np.linspace(x_min, x_max, n_points)[1:]
    func = ex03_wci.get_function_object()    
    values = [func(x) for x in grid]
    ax.plot(grid, values, color = 'red', linewidth = 2)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xlabel("a", fontsize = 15)
    ax.set_ylabel("total energy", fontsize = 15)
    
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.tick_params(axis='both', which='minor', labelsize=12)
    
ex03_plot = WidgetPlot(plot_total_energy, WidgetParbox(x_min = (0.1, 0, 1.0, 0.1, r'$x_{min}$'),
                                                        x_max = (4.0, 1.0, 10, 0.1, r'$x_{max}$'),
                                       y_min = (-1.0, -10, 0, 0.1, r'$y_{min}$'),
                                       y_max = (2.0, 0, 10, 0.1, r'$y_{max}$'),
                                       ));
ref_val = np.linspace(0.2, 5, 10)
ref_nrg = [3936501953.1249976,
 557.321470897532,
 -3.1707345581849036,
 -0.4858813626814373,
 -0.1047196369691888,
 -0.030580375933941077,
 -0.010997872358105667,
 -0.004589582164219913,
 -0.002140404149784412,
 -0.0010879339519999998]
ex03_ref_values = {(val,) : nrg for val, nrg in zip(ref_val, ref_nrg) }
ex03_wcc = WidgetCodeCheck(ex03_wci, ref_values = ex03_ref_values, demo=ex03_plot)    
display(ex03_wcc)

<span style="color:blue">**04** Is the equilibrium separation between the particles the same as that which minimizes the energy of a dimer? Write both the analytical expression, finding the minimum of the total energy that contains all interactions as a function of the square side $a$. Compare it with the plot above.</span>

In [ ]:
ex04_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex04-answer", ex04_txt, "value")
display(ex04_txt)

# Locality, cutoffs and minimum-image convention

One typical problem one encounters is that non-bonded potentials must (in principles) be evaluated among _all pairs of atoms_

$$
V = \frac{1}{2} \sum_{ij} v(|\mathbf{r}_{i}-\mathbf{r}_j|)
$$

This means that the computational effort grows as $N_{\text{atoms}}^2$ for a finite structure. But what about a _periodic_ structure? Then one would need to sum over multiple cells, making the effort essentially infinite!

In practice this is a real issue only for electrostatic interactions (for which [solutions](https://en.wikipedia.org/wiki/Ewald_summation) exist, but are much too complicated for this introductory course). For other long-range terms one usually artificially makes the interaction zero beyond a selected _cutoff_ distance $r_\text{cut}$. This is an approximation, but hardly the worst one we are making - the functional form of the potential is an approximation anyway. 

This below is a cluster of rare-gas atoms (which are well modeled by a LJ potential). If you click on an atom, it will highlight the atoms within the selected cutoff distance. Experiment with it to get a feel of the range of the interactions and how many atoms are actually included when you select different values for the cutoff. 

In [ ]:
lj55 = read('data/lj-structures.xyz',":1")

properties = {}
cs04 = chemiscope.show(lj55, mode="structure",                      
                     environments=chemiscope.all_atomic_environments(lj55),
                     settings={"structure":[{"bonds":True, "unitCell":False,
                                            "environments": {"cutoff": 3}}]}                    
                    )

def update_co(change):
    cs04.settings={"structure": [{"environments": {"cutoff": pb04.value['co']}}]}
pb04 = WidgetParbox(onchange=update_co, co=(3.,1,5,0.25, r"environment cutoff / Å"))
display(VBox([pb04,cs04]))

<span style="color:blue">**05** Write a function that loops over all pairs of atoms in this icosahedral cluster, and computes a LJ potential (with unit $\epsilon$ and $\sigma$). Only compute the potential for $r_{ij}<r_\mathrm{cut}$. Observe the plot demonstrating the convergence of the total energy. </span>

_NB: you'll have to exclude the i=j case, as the sum should extend over actual pairs_ 

In [ ]:
# set upt the code widget window
ex05_wci = WidgetCodeInput(
        function_name="total_LJ_icosahedral", 
        function_parameters="r_cut",
        docstring="""
Computes the total LJ energy of an icosahedral cluster, with a hard cutoff of the pair potential

:param r_cut: cutoff distance
        
:return: total LJ energy of the icosahedral cluster
""",
        function_body="""
        
import numpy as np
from ase.io import read
lj55 = read('data/lj-structures.xyz',0)
coordinates = lj55.positions

def compute_LJ(r): 
    # pair LJ potential for sigma=1, epsilon=1
    return -1  # change to actual value

total = 0.0 if r_cut<1 else -1
# write a loop accumulating the potential only for atoms that are closer than r_cut

return total
"""
        )

data_dump.register_field("ex05-function", ex05_wci, "function_body")

def plot_icosahedral_energy(ax, n_points = 100):
    x_max, y_min, y_max = 8, -2.8e2, -240
    grid = np.linspace(0, x_max, n_points)
    func = ex05_wci.get_function_object()    
    values = [func(x) for x in grid]
    
    if max(values)<y_min or min(values)>y_max:
        y_max = max(values)
        y_min = min(values)
    
    ax.plot(grid, values, color = 'red', linewidth = 2)
    ax.set_xlim(0, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xlabel(r"$r_{\mathrm{cut}}$ / Å", fontsize = 15)
    ax.set_ylabel("total energy", fontsize = 15)
    ax.set_title("icosahedral cluster")
    
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.tick_params(axis='both', which='minor', labelsize=12)
    
ex05_plot = WidgetPlot(plot_icosahedral_energy);


def match_energy(first, second, epsilon = 1e-5):
    return abs(first - second) < 1e-5

def reference_func_05(r_cut):
    lj55 = read('data/lj-structures.xyz',0)
    coordinates = lj55.positions
    def compute_LJ(r):
        return 4 * (1.0 / (r ** 12) - 1.0 / (r ** 6))
    
    def compute_distance(first, second):
        total = 0.0
        for i in range(3):
            total += (first[i] - second[i]) ** 2
        return np.sqrt(total)
    
    total = 0.0
    for i in range(len(coordinates)):
        for j in range(i + 1, len(coordinates)):
            distance = np.linalg.norm(coordinates[i]-coordinates[j])
            if distance < r_cut:
                total += compute_LJ(distance)
    return total

ex_05_ref_values = {(value, ) : reference_func_05(value)
                    for value in np.linspace(0.2, 10, 50)}

ex05_wcc = WidgetCodeCheck(ex05_wci, ref_values = ex_05_ref_values, demo=ex05_plot)       
display(ex05_wcc)

Now let's do the same for a bulk sample. Even if the cutoff is finite, one may have to sum over multiple copies of the supercell to account for all the interactions that contribute to the energy of the periodic solid. Ypu can get an idea of how the environments extend over multiple copies of the supercell here:

In [ ]:
lj55 = read('data/lj-structures.xyz',"1:")

properties = {}
cs05 = chemiscope.show(lj55, mode="structure",                      
                     environments=chemiscope.all_atomic_environments(lj55),
                     settings={"structure":[{"bonds":False, "unitCell":True,
                                            "supercell": {'0': 3, '1': 3, '2': 3},
                                            "environments": {"cutoff": 3}}]}                    
                    )

def update_co(change):
    cs05.settings={"structure": [{"environments": {"cutoff": pb05.value['co']}}]}
pb05 = WidgetParbox(onchange=update_co, co=(3.,1,5,0.25, r"environment cutoff / Å"))
display(VBox([pb05,cs05]))

If however the supercell is large enough to contain a sphere of size $r_\text{cut}$ (informally, if it is at least $2r_\text{cut}$ in every direction) one can utilize a more efficient scheme, the _minimum image convention_.

Essentially, one would only let the interaction loop run over the atoms in the supercell, and for each pair consider the periodic replica with the smallest possible separation. For an orthogonal box, this can be achieved by restricting each Cartesian coordinate of the interatomic separation _vectors_ to be between $-|\mathbf{a}_i|/2$ and $|\mathbf{a}_i|/2$. If `ax` is the side of the box along one dimension, the basic pattern is 

```
dx = x[i]-x[j]
dx /= ax
dx = ax*(dx-np.rint(dx))
```

<span style="color:blue">**06** Modify the function from Ex. 5 to compute the total energy for a bulk LJ structure. Only compute the potential for $r_{ij}<r_\mathrm{cut}$, but make sure you use the minimum image convention when computing the distance between atoms $i$ and $j$. Observe the plot demonstrating the convergence of the total energy. </span>

In [ ]:
# set upt the code widget window
ex06_wci = WidgetCodeInput(
        function_name="total_LJ_bulk", 
        function_parameters="r_cut",
        docstring="""
Computes the total LJ energy of aLJ bulk, with a hard cutoff of the pair potential

:param r_cut: cutoff distance
        
:return: total LJ energy of the icosahedral cluster
""",
        function_body="""
        
import numpy as np
from ase.io import read
ljbulk = read('data/lj-structures.xyz',1)
coordinates = ljbulk.positions

def compute_LJ(r): 
    # pair LJ potential for sigma=1, epsilon=1
    return -1  # change to actual value

total = 0.0
# write a loop accumulating the potential only for atoms that are closer than r_cut.
# remember to use the minimum image convention. What happens if you don't do that?

return total
"""
        )

data_dump.register_field("ex06-function", ex06_wci, "function_body")

def plot_bulk_energy(ax, n_points = 100):
    x_max, y_min, y_max = 8, -3.6e2, -320
    grid = np.linspace(0.1, x_max, n_points)
    func = ex06_wci.get_function_object()    
    values = [func(x) for x in grid]
    if max(values)<y_min or min(values)>y_max:
        y_max = max(values)
        y_min = min(values)
    
    ax.plot(grid, values, color = 'red', linewidth = 2)
    ax.set_xlim(0, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xlabel(r"$r_{\mathrm{cut}}$ / Å", fontsize = 15)
    ax.set_ylabel("total energy", fontsize = 15)
    ax.set_title("LJ bulk")
    
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.tick_params(axis='both', which='minor', labelsize=12)
    
ex06_plot = WidgetPlot(plot_bulk_energy);
cell_06 = read('data/lj-structures.xyz',1).cell
coordinates_06 = read('data/lj-structures.xyz',1).positions
def reference_func_06(r_cut):
    def compute_LJ(r):
        return (1.0 / (r ** 6) - 1.0) * 4.0 / (r ** 6)
    
    def compute_distance(first, second, a0):
        dx = (first-second)/a0
        dx=a0*(dx-np.rint(dx))
        return np.linalg.norm(dx)
    
    total = 0.0
    for i in range(len(coordinates_06)):
        for j in range(i + 1, len(coordinates_06)):
            distance = compute_distance(coordinates_06[i], coordinates_06[j], cell_06[0,0])
            if distance < r_cut:
                total += compute_LJ(distance)
    return total

ex_06_ref_values = {(value, ) : reference_func_06(value)
                    for value in np.linspace(0.5, 10, 10)}

ex06_wcc = WidgetCodeCheck(ex06_wci, ref_values = ex_06_ref_values, demo=ex06_plot)       
display(ex06_wcc)

# Using external potential calculators

Most of the time, you don't re-implement from scratch the calculation of an interatomic potential, but you use the subroutine provided by others. Unsurprisingly, there is a LJ calculator implemented in ASE. The usage is simple:

```python
from ase.calculators import lj
lj_calc = lj.LennardJones(sigma=1.0, epsilon=1.0, rc=2.0)

structure.calc = lj_calc   # `structure` here is an ase.Atoms object
energy = structure.get_potential_energy()  # computes the potential
```



There are important differences with the implementation you have just realized in the previous exercise:

1. The ASE implementation does not rely on a minimum image convention, so the energy can be defined meaningfully also for $r_{\mathrm{cut}}$ greater than half the supercell size
2. To avoid the discontinuity in the pair potential at $r_{\mathrm{cut}}$, the pair potential is re-defined as $v(r)\leftarrow v(r)-v(r_{\mathrm{cut}})$. This shifting procedure can be avoided using a _smooth cutoff_, which is activated by using the `smooth=True` option in the initialization of `lj.LennardJones`. Note that this option joins smoothly the LJ potential to zero starting at a position `ro` that is by default taken to be `rc*2/3`. If you want to replicate the abrupt truncation of the potential you used in your routine, you should also set `ro=rcut-1e-10` in the initialization. 

<span style="color:blue">**07** Write a function that uses the ASE calculator. When you move the slider, you will see the value of the energy obtained with your function (the one used in Ex. 6) and the one you compute here using the ASE calculator. Note the differences as you change cutoff and as you change the options to use a sharp truncation rather than a shifted potential. </span>

_NB: the code verification is built for the truncated potential so do not worry if the verification fails when you don't set `smooth=True, ro=rcut-1e-10`._

In [ ]:
# set upt the code widget window
ex07_wci = WidgetCodeInput(
        function_name="total_LJ_bulk_ase", 
        function_parameters="r_cut",
        docstring="""
Computes the total LJ energy of aLJ bulk, with a hard cutoff of the pair potential using ASE routines
 
:param r_cut: cutoff distance
        
:return: total LJ energy
""",
        function_body="""
      
from ase.io import read
ljbulk = read('data/lj-structures.xyz',1)

from ase.calculators import lj
lj_calc = ...

ljbulk.calc = lj_calc
energy = 0 

return energy
"""
        )


data_dump.register_field("ex07-function", ex07_wci, "function_body")

ex07_html = HTML(value=f"")

ex07_box = HBox(layout=Layout(height='250px', overflow_y='auto'))
ex07_box.children += (ex07_html,)

def ex07_update(args):
    import time
    
    r_cut = args['new']['r_cut']
        
    time_load = time.time()
    from ase.io import read
    ljbulk = read('data/lj-structures.xyz',1)
    time_load = time.time() - time_load

    time_init = time.time()
    from ase.calculators import lj
    lj_calc = lj.LennardJones(sigma=1.0, epsilon=1.0, rc=r_cut, ro=r_cut-1e-10, smooth = True)
    time_init = time.time() - time_init
    
    func = ex07_wci.get_function_object() 
    begin = time.time()
    ase_energy = func(r_cut)
    ase_time = time.time() - begin - time_load - time_init                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
    first = f"r_cut: {r_cut}<br> ase energy: {ase_energy:.2f}<br> ase time: {1000 * ase_time:.1f} miliseconds<br>"
    
    func = ex06_wci.get_function_object()
    begin = time.time()
    ex06_energy = func(r_cut)
    ex06_time = time.time() - begin - time_load
    
    second = f"ex06 energy: {ex06_energy:.2f}<br> ex06 time: {1000 * ex06_time:.1f} miliseconds<br>"
    ex07_html.value = first + second

def reference_func_07(r_cut):
    from ase.io import read
    ljbulk = read('data/lj-structures.xyz',1)
    
    from ase.calculators import lj
    lj_calc = lj.LennardJones(sigma=1.0, epsilon=1.0, rc=r_cut, ro=r_cut-1e-10, smooth = True)

    ljbulk.calc = lj_calc
    energy = ljbulk.get_potential_energy()

    return energy


parbox_ex07 = WidgetParbox(r_cut = (0.1, 0.1, 20.0, 0.1, r'$r_{cut}$'), onchange=ex07_update)

ex_07_ref_values = {(value, ) : reference_func_07(value)
                    for value in np.linspace(0.2, 5, 10)}

ex07_wcc = WidgetCodeCheck(ex07_wci, ref_values = ex_07_ref_values, demo = (parbox_ex07, ex07_box))
display(ex07_wcc)

# Forcefields are for forces!

The derivative of the potential with respect to atomic coordinates is (minus) the _force_ that acts on the atoms,

$$
\mathbf{f}_j = -\frac{\partial V(\{\mathbf{r}_i\})}{\partial \mathbf{r}_j}. 
$$

We will see, later in this course, how the force can be used to model the evolution of the atoms, and to find the most stable/energetically-favorable structures. 

For most potentials, the forces can be computed analytically at a small overhead with respect to the calculation of the energy. Forces can be obtained using the same calculator discussed in the previous section, by instrumenting a structure object by setting the `structure.calc` field, and then calling `structure.get_forces()`.
It is also possible to compute forces by _finite differences_, that is by applying the definition of the derivative of the energy as a limit. Rather than the usual definition, it is usually more stable to have a centered-difference expression

$$
\Delta V_{j\alpha}(h) = \frac{V(\{\mathbf{r}_i + h \delta_{ij} \hat{\mathbf{e}}_\alpha\}) 
- V(\{\mathbf{r}_i -h  \delta_{ij} \hat{\mathbf{e}}_\alpha\}) }{2h}
$$

where $V(\{\mathbf{r}_i + h \delta_{ij} \hat{\mathbf{e}}_\alpha\})$ indicates the energy computed for a selected atom $j$ moved by $h$ along the Cartesian coordinate $\alpha \in \{x,y,z\}$. The exact gradient is recovered by taking the limit for $h\rightarrow 0$.

<span style="color:blue">**08** How many energy evaluations would be needed to compute the force associated with a system with $N$ atoms using a centered-differences expression? </span>

In [ ]:
ex08_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex08-answer", ex08_txt, "value")
display(ex08_txt)

<span style="color:blue">**09** Write a function that constructs a LJ dimer, for a given interatomic separation $r$, and computes the force acting on one of the atoms by finite differences. For instance, the two atoms could be located at $(0,0,0)$ and at $(r,0,0)$. Compute the force by a finite displacement of the atom along the radial direction, and return its value. The magnitude of the displacement should be another parameter of the function. 
The reference value of the force is given by using the ASE built-in calculator, and is shown in the plot. 
</span>

In [ ]:
ex08_wci = WidgetCodeInput(
        function_name="get_force_numerical", 
        function_parameters="r, h",
        docstring="""
Computes force for the LJ dimer by finite differences
:param r: distance between the atoms
:param h: finite displacement
:return: Force acting on the atoms
""",
        function_body="""
        
from ase import Atoms
from ase.calculators import lj

plus =  ...
minus = ...
lj_calc = lj.LennardJones(sigma=1.0, epsilon=1.0, rc=r + 1.0 + h, ro=r + 1 - 1e-10 + h, smooth = True)

force = 0 
return force
"""
        )
data_dump.register_field("ex08-function", ex08_wci, "function_body")
def get_analytical(r):
    from ase import Atoms
    from ase.calculators import lj
    
    dimer = Atoms('HH', positions=[(0, 0, 0), (r, 0, 0)])
    lj_calc = lj.LennardJones(sigma=1.0, epsilon=1.0, rc=r + 1.0, ro=r + 1 - 1e-10, smooth = True) #epsilon is 1.0
    dimer.calc = lj_calc
    force = dimer.get_forces()[1][0]
    return force

def reference_func_08(r, h):
    from ase import Atoms
    from ase.calculators import lj
    
    first =  Atoms('HH', positions=[(0, 10, 10), (r - h, 10, 10)])
    second = Atoms('HH', positions=[(0, 0, -10-h), (r + h, 0, -10-h)])
    lj_calc = lj.LennardJones(sigma=1.0, epsilon=1.0, rc=r + 1.0 + h, ro=r + 1 - 1e-10 + h, smooth = True)
    first.calc = lj_calc
    second.calc = lj_calc
    
    first_energy = first.get_potential_energy()
    second_energy = second.get_potential_energy()
    force = (first_energy - second_energy) / (2 * h)
    return force

def plot_force(ax, h):
    epsilon = 1e-2
    grid = np.linspace(0, 3, 200)
    grid = [x for x in grid if x > (h + epsilon)]
    func = ex08_wci.get_function_object() 
    values = [func(r, h) for r in grid]
    
    analytical = [get_analytical(r) for r in grid]
   
    ax.plot(grid, analytical, color = 'blue', label = 'analytical')
    ax.plot(grid, values, color = 'red', label = 'finite differences')
    
    ax.legend()
    ax.set_xlabel("r", fontsize=15)
    ax.set_ylabel("force", fontsize=15)
    ax.set_ylim([-3, 3])
    ax.set_title(f"h={h}", fontsize =15)
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.tick_params(axis='both', which='minor', labelsize=12)
    
ex08_plot = WidgetPlot(plot_force, WidgetParbox(h = (0.1, 0.001, 1, 0.001, r'h')))
ex_08_ref_values = {}
for r in np.linspace(0.5, 3, 5):
    for h in np.linspace(0.01, 1, 5):
        ex_08_ref_values[r, h] = reference_func_08(r, h)
        
ex08_wcc = WidgetCodeCheck(ex08_wci, ref_values = ex_08_ref_values, demo=ex08_plot)       
display(ex08_wcc)

# Many body potentials

Many materials - most notably metals - are not well-described by pure pair interactions. One very successful approach to introduce effects that are "many body" in nature is given by the embedded atom models (EAMs): we will use an EAM potential for Al (you can see the [original publication](https://journals.aps.org/prb/pdf/10.1103/PhysRevB.59.3393) if you are curious), but first a little bit of theory.

EAMs assume that the total energy is the combination of a pair potential and an "embedding energy", 

$$
E_\mathrm{EAM} = \frac{1}{2}\sum_{ij} v(r_{ij}) + \sum_i F(\rho_i), \quad \rho_i = \sum_j \rho(r_{ij})
$$

The embedding energy is meant to model the non-additive effect of the accumulation of electron density in the vicinity of the $i$-th atom. Even though the expression for $E_\mathrm{EAM}$ is built from pair terms, the presence of the embedding function means that there is more to it. 

You can obtain an ASE calculator that evaluates the energy of a structure based on the EAM potential for Al from 
[Mishin et al.](https://journals.aps.org/prb/pdf/10.1103/PhysRevB.59.3393) by first loading the tabulated form

```python
from ase.calculators import eam
eamcalc = eam.EAM(potential='data/Al99.eam.alloy')
```

and then using it as the `.calc` member of a structure, as for the `LennardJones` calculator in the previous exercises. 

<span style="color:blue">**10** Write a function that constructs a dimer with bond length $a$, and a trimer with the structure of an equilateral triangle of side $a$. Based on the flag `eam` compute either a LJ potential (set $\sigma=2.6$, $\epsilon=0.4$) if it is `False`, or the Al EAM if it is `True`. Return the energy of the two structures and observe the plot of $E_\mathrm{trimer}$ and $3E_\mathrm{dimer}$.
</span>

In [ ]:
ex09_wci = WidgetCodeInput(
        function_name="get_energies", 
        function_parameters="a, eam_flag",
        docstring="""
Computes energies for dimer and trimer 

:param a: bond length
:param eam_flag: flag to compute eam or lj energy

:return: tuple with dimer and trimer energies
""",
        function_body="""
import ase
from ase.calculators import eam, lj
import numpy as np

if eam_flag:
    calc = ...
else:
    calc = ...

Al2 = ...
Al3 = ...

dimer_energy = 0
trimer_energy = 0.1

return dimer_energy, trimer_energy
"""
        )

data_dump.register_field("ex09-function", ex09_wci, "function_body")

def reference_func_09(a, eam_flag):
    import ase
    from ase.calculators import eam, lj
    import numpy as np

    Al3 = ase.Atoms("AlAl", positions=[[1,0,1], [1+a,0,1]])
    Al_dimer = ase.Atoms("AlAlAl", positions=[[1,0,0], [1+a,0,0], [1+a/2,a*np.sqrt(3)/2,0]])

    if eam_flag:
        calc = eam.EAM(potential='data/Al99.eam.alloy')
    else:
        calc = lj.LennardJones(sigma=2.6, epsilon=1.5, rc=a + 1.0, ro= a + 1.0 - 1e-10, smooth = True)

    Al_dimer.calc = calc
    Al3.calc = calc

    dimer_energy = Al_dimer.get_potential_energy()
    trimer_energy = Al3.get_potential_energy()

    return trimer_energy, dimer_energy

def plot_energies(ax, eam_flag):
    import numpy as np
    grid = np.linspace(0.5, 5, 50)
    func =  ex09_wci.get_function_object()
        
    curve = [func(a, eam_flag) for a in grid]
    dimer = [3*el[0] for el in curve]
    trimer = [el[1] for el in curve]
   
    ax.plot(grid, dimer, color = 'blue', label = r'$3E_\mathrm{dimer}$')
    ax.plot(grid, trimer, color = 'red', label = r'$E_\mathrm{trimer}$')
    
    ax.set_ylim(-6, 6)
    ax.set_xlabel("a / Å", fontsize = 15)
    ax.set_ylabel("energy / eV", fontsize = 15)
    if eam_flag:
        ax.set_title("EAM potential")
    else:
        ax.set_title("LJ potential")
    ax.legend()
    
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.tick_params(axis='both', which='minor', labelsize=12)    

ex09_plot = WidgetPlot(plot_energies, WidgetParbox(eam_flag=(False, "Use EAM")))
ex_09_ref_values = {}
for a in np.linspace(0.5, 5, 10):
    for eam_flag in [True, False]:
        ex_09_ref_values[a, eam_flag] = reference_func_09(a, eam_flag)
        
ex09_wcc = WidgetCodeCheck(ex09_wci, ref_values = ex_09_ref_values, demo=ex09_plot)       
display(ex09_wcc)


<span style="color:blue">**10** What do you observe as a function of dimer separation $a$? How can you explain this behavior? Can you create an arrangement of atoms that would allow you to perform a similar simple test with 4 particles? </span>

_Hint: <img src="figures/social-distancing.png" width="300"/>_

In [ ]:
ex10_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex10-answer", ex10_txt, "value")
display(ex10_txt)

# Fitting potentials

This far you have learned about some technical aspects of using interatomic potentials in an atomistic simulation. But what if you want to model a specific material, and don't have a potential you can fetch from the literature? The process of _fitting_ a potential so that it provides a realistic (or at least reasonable) description of a material is one of the most important (and painful) steps in atomic-scale modeling. 

The most naive approach involves tuning the parameters of the potential so that it matches the properties of a given system. Let's take as an example bulk aluminum. Aluminum has a cohesive energy of $E_0$ of 3.39 eV/atom, a _fcc_ lattice parameter at room temperature of $a_0$ 4.05 Å, and a bulk modulus $B_0$ of about 70 GPa. There are many physically motivated [equations of state](https://en.wikipedia.org/wiki/Birch%E2%80%93Murnaghan_equation_of_state) that one could derive for solids. Here we make the simple assumption that the energy of Al bulk as a function of lattice parameter is a quadratic expression, consistent with these constraints. Cohesive energy and lattice parameter imply that the energy as a function of lattice parameter should have the form 

$$
E(a) \approx E_0 + \frac{k}{2} (a-a_0)^2
$$ 

We only need to link $k$ with the bulk modulus.
The bulk modulus is [defined](https://en.wikipedia.org/wiki/Bulk_modulus) as $B_0=-V \partial p/\partial V$, where $p=-\partial E(V)/\partial V$ is in turn the pressure. You can check rather easily by performing a change of variables that this implies that $k=9B_0 a_0$.

<span style="color:blue">**11** This widget plots the energy versus lattice-parameter curve for Al.
The black curve corresponds to the parabolic shape inferred from experimental parameters, the blue dots to the values computed for the EAM potential, and the red dots to the points computed for a LJ potential with the $\sigma$ and $\epsilon$ parameters specified by the sliders. Adjust the sliders until you find the best match to the experimental curve. Write the parameters in the text box below, and comment on what you observe.
Can you match simultaneously $E_0$, $a_0$ and $B_0$? How does the EAM potential perform? What can you infer in terms of the difficulty in fitting empirical interatomic potentials?
</span>

In [ ]:
fcc_pos = np.asarray( [[0,0,0],[0.5,0.5,0],[0.5,0,0.5],[0,0.5,0.5]] ) 

E0 = -3.36 * 4 # takes into account that there are 4 atoms in the cubic box
a0 = 4.05
k = 70 * 0.0062415091 *9 *a0 # converts to GPa to eV/Á^3
ljcalc = lj.LennardJones(sigma=2.6, epsilon=0.395, rc=5*2.5)
eamcalc = eam.EAM(potential='data/Al99.eam.alloy')
def pot_fcc(a0, calc):
    struc = ase.Atoms("Al4", positions=fcc_pos*a0, cell=[a0,a0,a0], pbc=True)    
    struc.calc = calc
    return struc.get_potential_energy()

agrid = np.linspace(a0*0.9,a0*1.1,20)
eamgrid = [ pot_fcc(a, eamcalc) for a in agrid ]

def mkplot(ax, sigma, epsilon):
    ljcalc = lj.LennardJones(sigma=sigma, epsilon=epsilon, rc=4*sigma)
    ljgrid = [ pot_fcc(a, ljcalc) for a in agrid ]
    ax.plot(agrid, E0+0.5*k*(agrid-a0)**2, 'k--', label='Exp.')
    ax.plot(agrid, eamgrid, 'b.', label="EAM")
    ax.plot(agrid, ljgrid, 'r.', label="LJ fit")
    ax.legend()
    ax.set_ylim(min(min(ljgrid), min(eamgrid)), max(max(eamgrid), np.mean(ljgrid)))
    ax.set_xlabel(r"$r$ / Å")
    ax.set_ylabel(r"$E$ / eV/cell")
pb10 = WidgetParbox(sigma=(3.0,2.0,4.0,0.01,r"$\sigma$ / Å", dict(readout_format='.2f') ), 
                                            epsilon=(0.5,0.3,0.6,0.0001,r"$\epsilon$ / eV/cell", dict(readout_format='.3f') ))
data_dump.register_field("ex10-values-sigma", pb10._controls["sigma"], "value")
data_dump.register_field("ex10-values-epsilon", pb10._controls["epsilon"], "value")
pp10 = WidgetPlot(mkplot, parbox=pb10 )
display(pp10)

In [ ]:
ex11_txt = Textarea("enter the best fit parameters, and your considerations", layout=Layout(width="100%"))
data_dump.register_field("ex11-answer", ex11_txt, "value")
display(ex11_txt)